# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [186]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/lemonanaquis/anaconda3/envs/pr_robocup/lib/python3.13/site-packages/IPython/core/magics/pylab.py:166: UserWarning: pylab import has clobbered these variables: ['permutation', 'clf']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [187]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['HeadBack', 'Sit', 'Stand', 'Left', 'Right', 'Crouch', 'Back', 'StandInit', 'Frog', 'Knee', 'Belly']


In [378]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    #classes = listdir(ROBOT_POSE_DATA_DIR)
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [379]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
#print(type(all_data), type(all_target))
for i in range(len(classes)):
    data, target = load_pose_data(i)
    #print(classes[i])
    #print(target)
    all_data.extend(np.array(data))
    all_target.extend(np.array(target))
    #print('all_data', all_data)
all_data = np.array(all_data)
print(all_data)

all_target = np.array(all_target)

print('all_target', all_data.shape)
#print('all_target', all_data)
print ('total number of data', len(all_data))

[[ 0.20559788 -0.2392621  -0.24386406 ... -0.13341594 -3.05432558
  -0.02094412]
 [ 0.20559788 -0.2392621  -0.24386406 ... -0.13341594 -3.05432558
  -0.02094412]
 [ 0.20559788 -0.2392621  -0.24386406 ... -0.13341594 -3.05432558
  -0.02094412]
 ...
 [ 0.          0.          0.         ...  0.          0.
   1.57079633]
 [ 0.          0.          0.         ...  0.          0.
   1.57079633]
 [ 0.          0.          0.         ...  0.          0.
   1.57079633]]
all_target (222, 10)
total number of data 222


In [380]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [381]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [382]:
# YOUR CODE HERE

clf.fit(all_data[training_data], all_target[training_data])
all_data = np.reshape(all_data,(-1,10))
#all_data = np.reshape(-1, 10)
#all_target = np.reshape(-1,)
print(type(all_data[-1]))
#print(all_target.shape)
print(type(all_data))
print(all_data[-1].shape)
print(all_data.shape)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(10,)
(222, 10)


### predicting

In [383]:
clf.predict(all_data[-1:]), all_target[-1:]
print(clf.predict(all_data[-1:]))

[10]


In [384]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [385]:
expected = []
predicted = []
# YOUR CODE HERE

expected = np.array(expected)
predicted = np.array(predicted)


predicted = (clf.predict(all_data[training_data]))
expected = all_target[training_data]

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        17
           2       1.00      1.00      1.00         9
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00        20
           6       1.00      1.00      1.00        15
           7       1.00      1.00      1.00        38
           8       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00        13

    accuracy                           1.00       155
   macro avg       1.00      1.00      1.00       155
weighted avg       1.00      1.00      1.00       155


Confusion matrix:
[[ 8  0  0  0  0  0  0  0  0  0  0]
 [ 0 17  0  0  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  0  0  0  0  0  0]
 [ 0  0  0 15  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [386]:
expected = []
predicted = []
# YOUR CODE HERE

test_data = permutation[n_training_data:]

expected = np.array(expected)
predicted = np.array(predicted)


predicted = (clf.predict(all_data[test_data]))
expected = all_target[test_data]

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         5
           4       1.00      0.80      0.89         5
           5       0.91      1.00      0.95        10
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        14
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00         6

    accuracy                           0.99        67
   macro avg       0.99      0.98      0.99        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 2  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [389]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [388]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte